In [10]:
import json
import requests
from time import sleep
from bs4 import BeautifulSoup

sess = requests.Session()

search_page = sess.get("https://www.ims.tau.ac.il/Tal/KR/Search_P.aspx").text
search_soup = BeautifulSoup(search_page, features="lxml")

In [11]:
all_departments = search_soup.select(".table1 select.freeselect.list")
all_options = []

for department in all_departments:
    print("Department:")
    options = [x for x in department.select("option") if x.text != ""]
    if options[0].text.startswith("כל "):
        options = [options[0]]

    for option in options:
        print(option.text)

    all_options.append((department["name"], [option["value"] for option in options]))

print(all_options)

Department:
כל הפקולטה לאמנויות
Department:
כל הפקולטה להנדסה
Department:
כל הפקולטה למדעי החברה
Department:
כל הפקולטה למדעי החיים
Department:
כל הפקולטה למדעי הרוח
Department:
כל הפקולטה למדעים מדויקים
Department:
כל הפקולטה למשפטים
Department:
כל הפקולטה לניהול
Department:
כל הפקולטה לרפואה
Department:
ביה"ס לעבודה סוציאלית
ביה"ס לחינוך
 - חינוך מיוחד וייעוץ חינוכי
 - מדיניות ומינהל בחינוך
 - תכנון לימודים והוראה
 - חינוך מתמטי, מדעי וטכנולגי
ביה"ס למדעי המוח
Department:
אנגלית כשפה זרה
שפות זרות
Department:
כל הקורסים בכלים שלובים
Department:
כל הקורסים בלימודי סייבר
Department:
כל הקורסים
[('lstDep1', ['08']), ('lstDep2', ['05']), ('lstDep3', ['10']), ('lstDep4', ['04']), ('lstDep5', ['06-16']), ('lstDep6', ['03']), ('lstDep7', ['14']), ('lstDep8', ['12']), ('lstDep9', ['01']), ('lstDep10', ['11', '07', '0721-0712-0713', '0716-0722-0734-0732', '0731-0733-0738-0756-0752-0754', '0751-0755-0757', '15']), ('lstDep11', ['2171', '2172']), ('lstDep12', ['1880-1882']), ('lstDep13', ['1843

In [12]:
# Create base query
base_fields = {"lstYear1": "2021", "txtShemKurs": "", "txtShemMore": ""}
for option in all_options:
    base_fields[option[0]] = ""

queries = []

# Create query for every option
for select in all_options:
    for select_option in select[1]:
        new_fields = base_fields.copy()
        new_fields[select[0]] = select_option
        queries.append(new_fields)

queries = [queries[5]]  # מדעים מדויקים

In [13]:
def parse_result_page(result_soup):
    all_rows = result_soup.select_one("#frmgrid table[dir=rtl]").select("tr")
    pagenum_row = all_rows[0]
    all_rows = all_rows[1:]
    i = 0
    courses = []
    while i < len(all_rows):
        try:
            if "kotcol" in all_rows[i]["class"] and len(list(all_rows[i].children)) == 2:
                course = {}
                courses.append(course)
                # start of course
                i += 1
                # next row is the course name + id
                course["name"] = list(all_rows[i].children)[1].text
                course["id"] = next(next(all_rows[i].children).children).strip()
                course["group"] = all_rows[i].select_one("span").next_element.next_element.strip()
                i += 1
                # this row has the faculty
                course["faculty"] = list(all_rows[i].children)[1].text
                i += 1
                # look for the times and instructor section
                while not ("kotcol" in all_rows[i - 1]["class"]):
                    i += 1
                lecturer = None

                lessons = []
                course["lessons"] = lessons
                while len(list(all_rows[i].children)) == 7:
                    curr_lecturer, ofen_horaa, building, room, day, time, semester = [x.text.strip() for x in all_rows[i].children]
                    if lecturer == None and len(curr_lecturer) != 0:
                        lecturer = curr_lecturer
                        lecturer_found = True
                    lessons.append({"ofen_horaa": ofen_horaa, "building": building, "room": room, "day": day, "time": time, "semester": semester})
                    i += 1

                course["lecturer"] = lecturer
                print(course)
            else:
                i += 1
        except KeyError:
            i += 1

    return courses

In [14]:
headers = {"Accept": "text/html,application/xhtml+xml,application/xml", "Content-Type": "application/x-www-form-urlencoded", "User-Agent": "CourseScrape"}
print(queries)

courses = []
for query in queries:
    print("Sleeping...")
    sleep(5)
    print("Requesting...", end="")
    result_soup = BeautifulSoup(sess.post("https://www.ims.tau.ac.il/Tal/KR/Search_L.aspx", data=query, headers=headers).text, features="lxml")
    print("Done!")
    try:
        courses += parse_result_page(result_soup)
    except:
        continue  # probably a wrong page
    pagenum = 1

    # go to next pages
    while len(result_soup.select("#next")) > 0:
        pagenum += 1
        # get the "dir1": "1" is for the next button
        next_page_query = {}
        for hidden_input in result_soup.select("input[type=hidden]"):
            try:
                next_page_query[hidden_input["name"]] = hidden_input["value"]
            except KeyError:
                pass

        next_page_query["dir1"] = "1"
        print("Sleeping...")
        sleep(5)
        print(f"Requesting... (Page {pagenum})", end="")
        result_soup = BeautifulSoup(sess.post("https://www.ims.tau.ac.il/Tal/KR/Search_L.aspx", data=next_page_query, headers=headers).text, features="lxml")
        print("Done!")
        courses += parse_result_page(result_soup)

[{'lstYear1': '2021', 'txtShemKurs': '', 'txtShemMore': '', 'lstDep1': '', 'lstDep2': '', 'lstDep3': '', 'lstDep4': '', 'lstDep5': '', 'lstDep6': '03', 'lstDep7': '', 'lstDep8': '', 'lstDep9': '', 'lstDep10': '', 'lstDep11': '', 'lstDep12': '', 'lstDep13': '', 'lstDep14': ''}]
Sleeping...
Requesting...Done!
{'name': 'בטיחות כללית וחשמל', 'id': '0300-5000', 'group': '01', 'faculty': 'מדעים מדויקים/פקולטה למדעים מדויקים', 'lessons': [{'ofen_horaa': 'הדרכה אישית', 'building': '', 'room': '', 'day': '', 'time': '', 'semester': "א'"}], 'lecturer': None}
{'name': 'בטיחות כללית וחשמל', 'id': '0300-5000', 'group': '02', 'faculty': 'מדעים מדויקים/פקולטה למדעים מדויקים', 'lessons': [{'ofen_horaa': 'הדרכה אישית', 'building': '', 'room': '', 'day': '', 'time': '', 'semester': "ב'"}], 'lecturer': None}
{'name': 'עזרה ראשונה', 'id': '0300-5001', 'group': '01', 'faculty': 'מדעים מדויקים/פקולטה למדעים מדויקים', 'lessons': [{'ofen_horaa': 'הדרכה אישית', 'building': '', 'room': '', 'day': '', 'time': ''

Requesting... (Page 2)Done!
{'name': "מעבדה בפיזיקה ג -א' תלמידי תכנית המצטיינים - אנגלית", 'id': '0321-3308', 'group': '01', 'faculty': 'מדעים מדויקים/פיזיקה', 'lessons': [{'ofen_horaa': 'מעבדה', 'building': 'פיזיקה-שנקר', 'room': '109', 'day': 'א', 'time': '09:00-12:00', 'semester': "א'"}], 'lecturer': "פרופ'\xa0בק-סבטיצקי\xa0שרה חיה"}
{'name': "מעבדה בפיזיקה ג' - ב' לתלמידי תכנית המצטיינים - אנגלית", 'id': '0321-3309', 'group': '01', 'faculty': 'מדעים מדויקים/פיזיקה', 'lessons': [{'ofen_horaa': 'מעבדה', 'building': '', 'room': '', 'day': 'ב', 'time': '09:00-12:00', 'semester': "ב'"}, {'ofen_horaa': 'מעבדה', 'building': '', 'room': '', 'day': 'ב', 'time': '13:00-16:00', 'semester': "ב'"}, {'ofen_horaa': 'מעבדה', 'building': '', 'room': '', 'day': 'ד', 'time': '09:00-12:00', 'semester': "ב'"}, {'ofen_horaa': 'מעבדה', 'building': '', 'room': '', 'day': 'ד', 'time': '13:00-16:00', 'semester': "ב'"}, {'ofen_horaa': 'מעבדה', 'building': '', 'room': '', 'day': '', 'time': '', 'semester': "

Requesting... (Page 3)Done!
{'name': 'ישומי למידת מכונה במדעי כדור הארץ', 'id': '0341-4126', 'group': '01', 'faculty': 'מדעים מדויקים/גאופיזיקה', 'lessons': [{'ofen_horaa': 'שיעור', 'building': '', 'room': '', 'day': 'ג', 'time': '16:00-19:00', 'semester': "ב'"}], 'lecturer': 'ד"ר\xa0סגל רוזנהי\xa0מיכל'}
{'name': 'מיגרציה סייסמית', 'id': '0341-4192', 'group': '01', 'faculty': 'מדעים מדויקים/גאופיזיקה', 'lessons': [{'ofen_horaa': 'שיעור ותרגיל', 'building': 'קפלון', 'room': '205', 'day': 'ד', 'time': '13:00-15:00', 'semester': "א'"}], 'lecturer': "פרופ'\xa0רשף\xa0משה"}
{'name': 'סמינריון מחלקתי (קולקוויום)', 'id': '0341-4210', 'group': '01', 'faculty': 'מדעים מדויקים/גאופיזיקה', 'lessons': [{'ofen_horaa': 'קולוקויום', 'building': 'אודי.הולצבלט', 'room': '007', 'day': 'ב', 'time': '11:00-12:00', 'semester': "א'"}], 'lecturer': 'ד"ר\xa0ברקן\xa0רועי'}
{'name': 'סמינריון מחלקתי קולוקויום', 'id': '0341-4213', 'group': '01', 'faculty': 'מדעים מדויקים/גאופיזיקה', 'lessons': [{'ofen_horaa': 'קו

Requesting... (Page 4)Done!
{'name': 'תרמודינמיקה', 'id': '0351-2202', 'group': '01', 'faculty': 'מדעים מדויקים/כימיה', 'lessons': [{'ofen_horaa': 'שיעור', 'building': 'אודיטו. מלמד', 'room': '006', 'day': 'א', 'time': '12:00-14:00', 'semester': "א'"}, {'ofen_horaa': 'שיעור', 'building': 'אודיטו. מלמד', 'room': '006', 'day': 'ה', 'time': '12:00-14:00', 'semester': "א'"}], 'lecturer': "פרופ'\xa0הוד\xa0עודד"}
{'name': 'תרמודינמיקה', 'id': '0351-2202', 'group': '02', 'faculty': 'מדעים מדויקים/כימיה', 'lessons': [{'ofen_horaa': 'תרגיל', 'building': 'אורנשטיין', 'room': '102', 'day': 'א', 'time': '14:00-16:00', 'semester': "א'"}], 'lecturer': None}
{'name': 'תרמודינמיקה', 'id': '0351-2202', 'group': '03', 'faculty': 'מדעים מדויקים/כימיה', 'lessons': [{'ofen_horaa': 'תרגיל', 'building': 'אורנשטיין', 'room': '110', 'day': 'ב', 'time': '12:00-14:00', 'semester': "א'"}], 'lecturer': None}
{'name': 'תרמודינמיקה', 'id': '0351-2202', 'group': '04', 'faculty': 'מדעים מדויקים/כימיה', 'lessons': [{'o

Requesting... (Page 5)Done!
{'name': 'מבוא למחשבים לסטטיסטיקאים', 'id': '0365-1800', 'group': '01', 'faculty': 'מדעים מדויקים/סטטיסטיקה וחקר ביצועים', 'lessons': [{'ofen_horaa': 'שיעור', 'building': "צ'ק פוינט", 'room': '002', 'day': 'ג', 'time': '15:00-18:00', 'semester': "א'"}], 'lecturer': "פרופ'\xa0גורפיין אורגד\xa0מלכה"}
{'name': 'מבוא למחשבים לסטטיסטיקאים', 'id': '0365-1800', 'group': '02', 'faculty': 'מדעים מדויקים/סטטיסטיקה וחקר ביצועים', 'lessons': [{'ofen_horaa': 'תרגיל', 'building': 'פיזיקה-שנקר', 'room': '204', 'day': 'א', 'time': '16:00-17:00', 'semester': "א'"}], 'lecturer': 'מר\xa0נוי\xa0איל'}
{'name': 'מבוא למחשבים לסטטיסטיקאים', 'id': '0365-1800', 'group': '03', 'faculty': 'מדעים מדויקים/סטטיסטיקה וחקר ביצועים', 'lessons': [{'ofen_horaa': 'תרגיל', 'building': 'אודי.הולצבלט', 'room': '007', 'day': 'ב', 'time': '15:00-16:00', 'semester': "א'"}], 'lecturer': 'מר\xa0נוי\xa0איל'}
{'name': 'מבוא לסטטיסטיקה', 'id': '0365-1813', 'group': '01', 'faculty': 'מדעים מדויקים/סטטיסטי

Requesting... (Page 6)Done!
{'name': 'אלגברה לינארית 2א', 'id': '0366-1112', 'group': '01', 'faculty': 'מדעים מדויקים/מתמטיקה', 'lessons': [{'ofen_horaa': 'שיעור', 'building': 'ולפסון הנדסה', 'room': '001', 'day': 'ב', 'time': '10:00-12:00', 'semester': "א'"}, {'ofen_horaa': 'שיעור', 'building': "אודיטור' לב", 'room': '009', 'day': 'ה', 'time': '08:00-10:00', 'semester': "א'"}], 'lecturer': "פרופ'\xa0גינסבורג\xa0דוד"}
{'name': 'אלגברה לינארית 2א', 'id': '0366-1112', 'group': '02', 'faculty': 'מדעים מדויקים/מתמטיקה', 'lessons': [{'ofen_horaa': 'תרגיל', 'building': 'שרייבר מתמטי', 'room': '006', 'day': 'ג', 'time': '10:00-12:00', 'semester': "א'"}], 'lecturer': "מר\xa0סיניצ'קין\xa0אוריאל"}
{'name': 'אלגברה לינארית 2א', 'id': '0366-1112', 'group': '03', 'faculty': 'מדעים מדויקים/מתמטיקה', 'lessons': [{'ofen_horaa': 'שיעור', 'building': '', 'room': '', 'day': 'ג', 'time': '16:00-18:00', 'semester': "ב'"}, {'ofen_horaa': 'שיעור', 'building': '', 'room': '', 'day': 'ה', 'time': '15:00-17:00'

Requesting... (Page 7)Done!
{'name': 'חשבון דיפרנציאלי ואינטגרלי 4', 'id': '0366-2180', 'group': '04', 'faculty': 'מדעים מדויקים/מתמטיקה', 'lessons': [{'ofen_horaa': 'שיעור', 'building': '', 'room': '', 'day': 'ב', 'time': '15:00-17:00', 'semester': "ב'"}, {'ofen_horaa': 'שיעור', 'building': '', 'room': '', 'day': 'ד', 'time': '13:00-14:00', 'semester': "ב'"}], 'lecturer': "פרופ'\xa0בוחובסקי\xa0לב"}
{'name': 'חשבון דיפרנציאלי ואינטגרלי 4', 'id': '0366-2180', 'group': '05', 'faculty': 'מדעים מדויקים/מתמטיקה', 'lessons': [{'ofen_horaa': 'תרגיל', 'building': '', 'room': '', 'day': 'ב', 'time': '14:00-15:00', 'semester': "ב'"}], 'lecturer': 'מר\xa0סגל\xa0עמנואל'}
{'name': 'חשבון דיפרנציאלי ואינטגרלי 4', 'id': '0366-2180', 'group': '06', 'faculty': 'מדעים מדויקים/מתמטיקה', 'lessons': [{'ofen_horaa': 'תרגיל', 'building': '', 'room': '', 'day': 'ד', 'time': '18:00-19:00', 'semester': "ב'"}], 'lecturer': 'מר\xa0סגל\xa0עמנואל'}
{'name': 'לוגיקה', 'id': '0366-2194', 'group': '01', 'faculty': 'מד

Requesting... (Page 8)Done!
{'name': 'מתמטיקה בדידה', 'id': '0368-1118', 'group': '15', 'faculty': 'מדעים מדויקים/מדעי המחשב', 'lessons': [{'ofen_horaa': 'תרגיל', 'building': '', 'room': '', 'day': 'ה', 'time': '10:00-12:00', 'semester': "ב'"}], 'lecturer': 'מר\xa0לוי\xa0ניסן'}
{'name': 'מתמטיקה בדידה', 'id': '0368-1118', 'group': '16', 'faculty': 'מדעים מדויקים/מדעי המחשב', 'lessons': [{'ofen_horaa': 'תרגיל', 'building': '', 'room': '', 'day': 'ה', 'time': '14:00-16:00', 'semester': "ב'"}], 'lecturer': 'מר\xa0לוי\xa0ניסן'}
{'name': 'מתמטיקה בדידה', 'id': '0368-1118', 'group': '90', 'faculty': 'מדעים מדויקים/מדעי המחשב', 'lessons': [{'ofen_horaa': 'שיעור', 'building': '', 'room': '', 'day': '', 'time': '', 'semester': "ב'"}], 'lecturer': None}
{'name': 'הסתברות וסטטיסטיקה (לדו-חוגי)', 'id': '0368-2002', 'group': '01', 'faculty': 'מדעים מדויקים/מדעי המחשב', 'lessons': [{'ofen_horaa': 'שיעור', 'building': "צ'ק פוינט", 'room': '001', 'day': 'א', 'time': '10:00-13:00', 'semester': "א'"}], 

Requesting... (Page 9)Done!
{'name': 'מבוא לאבטחת מידע', 'id': '0368-3065', 'group': '02', 'faculty': 'מדעים מדויקים/מדעי המחשב', 'lessons': [{'ofen_horaa': 'תרגיל', 'building': '', 'room': '', 'day': 'א', 'time': '16:00-17:00', 'semester': "ב'"}], 'lecturer': 'מר\xa0איטקין\xa0ברק'}
{'name': 'מבוא לאבטחת מידע', 'id': '0368-3065', 'group': '03', 'faculty': 'מדעים מדויקים/מדעי המחשב', 'lessons': [{'ofen_horaa': 'תרגיל', 'building': '', 'room': '', 'day': 'א', 'time': '17:00-18:00', 'semester': "ב'"}], 'lecturer': 'מר\xa0איטקין\xa0ברק'}
{'name': 'למידה ממוחשבת מחיזוקים', 'id': '0368-3075', 'group': '01', 'faculty': 'מדעים מדויקים/מדעי המחשב', 'lessons': [{'ofen_horaa': 'שיעור', 'building': '', 'room': '', 'day': 'א', 'time': '12:00-15:00', 'semester': "ב'"}], 'lecturer': "פרופ'\xa0מנצור\xa0ישי"}
{'name': 'למידה ממוחשבת מחיזוקים', 'id': '0368-3075', 'group': '02', 'faculty': 'מדעים מדויקים/מדעי המחשב', 'lessons': [{'ofen_horaa': 'תרגיל', 'building': '', 'room': '', 'day': 'א', 'time': '15:

Requesting... (Page 10)Done!
{'name': 'סדנה ביישומי למידת מכונה לגרפיקה ממוחשבת', 'id': '0368-3533', 'group': '02', 'faculty': 'מדעים מדויקים/מדעי המחשב', 'lessons': [{'ofen_horaa': 'מעבדה', 'building': '', 'room': '', 'day': 'א', 'time': '08:00-11:00', 'semester': "א'"}], 'lecturer': 'ד"ר\xa0ברמנו\xa0עמית חיים'}
{'name': 'סדנה ביישומי למידת מכונה לגרפיקה ממוחשבת', 'id': '0368-3533', 'group': '03', 'faculty': 'מדעים מדויקים/מדעי המחשב', 'lessons': [{'ofen_horaa': 'מעבדה', 'building': '', 'room': '', 'day': 'ה', 'time': '12:00-15:00', 'semester': "א'"}], 'lecturer': None}
{'name': 'סדנה במבני נתונים לחיפוש במסדי נתונים יעילים', 'id': '0368-3534', 'group': '01', 'faculty': 'מדעים מדויקים/מדעי המחשב', 'lessons': [{'ofen_horaa': 'סדנה', 'building': '', 'room': '', 'day': 'ה', 'time': '14:00-16:00', 'semester': "ב'"}], 'lecturer': 'ד"ר\xa0מוריסון\xa0אדם'}
{'name': 'סדנה במבני נתונים לחיפוש במסדי נתונים יעילים', 'id': '0368-3534', 'group': '02', 'faculty': 'מדעים מדויקים/מדעי המחשב', 'lesson

In [15]:
with open("out.json", "w") as out_file:
    json.dump(courses, out_file, separators=(',', ':'), ensure_ascii=False, sort_keys=True)
print("DONNNEEEEE!")

DONNNEEEEE!


In [16]:
print(len(courses))

984
